In [1]:
%load_ext autoreload

In [2]:
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.multibody.parsing import Parser
import os
from pydrake.all import (
    LoadModelDirectives, ProcessModelDirectives, RevoluteJoint, 
    RationalForwardKinematics, CspaceFreePolytope, SeparatingPlaneOrder,
    RigidTransform, RotationMatrix, Rgba
)
import numpy as np
# from pydrake.geometry.optimization_dev import (CspaceFreePolytope, SeparatingPlaneOrder)
from iris_plant_visualizer import IrisPlantVisualizer
from pydrake.geometry import Role
from pydrake.geometry.optimization import IrisOptions, HPolyhedron, Hyperellipsoid, IrisInRationalConfigurationSpace
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver
import time

In [3]:
#construct our robot
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)

parser.package_map().Add("ciris_pgd", os.path.abspath(''))

directives_file = "/home/sgrg/rlg/SUPERUROP/ciris/models/bimanual_two_DOF.yaml"
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)


# oneDOF_iiwa_file = "/home/sgrg/rlg/SUPERUROP/ciris/assets/oneDOF_iiwa7_with_box_collision.sdf"
# with open(oneDOF_iiwa_file, 'r') as f:
#     oneDOF_iiwa_string = f.read()
# box_asset_file = "/home/sgrg/rlg/SUPERUROP/ciris/assets/box_small.urdf"
# with open(box_asset_file, 'r') as f:
#     box_asset_string = f.read()

In [4]:
plant.Finalize()

In [5]:
idx = 0
q0 = [0.0, 0.0]
val = 1.7
q_low  = np.array([-val, -val])
q_high = np.array([val, val])
# set the joint limits of the plant
# for model in models:
for joint_index in plant.GetJointIndices():
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        print("joint index: ", joint_index, "joint name: ", joint.name(), "IDX", idx)
        joint.set_default_angle(q0[idx])
        joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
        idx += 1

joint index:  JointIndex(6) joint name:  iiwa_joint_2 IDX 0
joint index:  JointIndex(14) joint name:  iiwa_joint_2 IDX 1


In [6]:
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True

# The object we will use to perform our certification.
cspace_free_polytope = CspaceFreePolytope(plant, scene_graph, SeparatingPlaneOrder.kAffine, q_star)

In [7]:
# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity if you want
# to see the plant with the collision geometries.
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, cspace_free_polytope, rational_forward_kin=Ratfk ,viz_role=Role.kProximity)
visualizer.visualize_collision_constraint(factor = 1.2, num_points = 100)
visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat_task_space.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))
visualizer.meshcat_nontc.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))

INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Meshcat listening for connections at http://localhost:7001
INFO:drake:Meshcat listening for connections at http://localhost:7002


In [8]:
import ipywidgets as widgets
from functools import partial

sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_q(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7)

In [9]:
# Some seedpoints
seed_points_q = np.array([   
                              [0.0, 0],
                              [0.4, 0.4],
                              [-0.5, 0.5],
                              [0.5,1.08],
                              [1.06, -0.48]
                              ])

seed_points = np.array([Ratfk.ComputeSValue(seed_points_q[idx], np.zeros((2,)))\
                        for idx in range(seed_points_q.shape[0])])
start = seed_points[0]
end = seed_points[-1]

visualizer.plot_cspace_points(seed_points, "/iris_seed_points", radius = 0.02)
visualizer.plot_nontc_points(seed_points_q, "/iris_seed_points_q", radius = 0.02)

In [10]:
default_scale = 1e-2
L1_ball = HPolyhedron.MakeL1Ball(2)
Linf_ball = HPolyhedron.MakeBox(-np.ones(2), np.ones(2))

template_C = np.vstack([L1_ball.A(), Linf_ball.A()])
template_d = np.hstack([default_scale*L1_ball.b(), default_scale/np.sqrt(2)*Linf_ball.b()])


def make_default_polytope_at_point(seed_point):
    return HPolyhedron(template_C, template_d + template_C @ seed_point)


# colors to plot the region.
default_alpha = 0.2
colors_dict = {
    0: Rgba(0.565, 0.565, 0.565, default_alpha), # gray
    1: Rgba(0.118, 0.533, 0.898, default_alpha), # bluish
    2: Rgba(1,     0.757, 0.027, default_alpha), # gold
    3: Rgba(0,     0.549, 0.024, default_alpha), # green   
    4: Rgba(0.055, 0.914, 0.929, default_alpha), # teal 
}

initial_regions = [(make_default_polytope_at_point(s), colors_dict[i]) for i, s in enumerate(seed_points)]
visualizer.add_group_of_regions_to_visualization(initial_regions, "initial_regions",
                            wireframe = False,
                           opacity = default_alpha)
visualizer.show_res_q(q_star)

In [11]:
q_star

array([0., 0.])

In [12]:
# set up the certifier and the options for different search techniques
solver_options = SolverOptions()
# set this to 1 if you would like to see the solver output in terminal.
solver_options.SetOption(CommonSolverOption.kPrintToConsole, 0)

os.environ["MOSEKLM_LICENSE_FILE"] = "/home/sgrg/mosek.lic"
with open(os.environ["MOSEKLM_LICENSE_FILE"], 'r') as f:
    contents = f.read()
    mosek_file_not_empty = contents != ''
print(mosek_file_not_empty)

solver_id = MosekSolver.id() if MosekSolver().available() and mosek_file_not_empty else ScsSolver.id()


solver_id = MosekSolver.id() if MosekSolver().available() else ScsSolver.id()

True


In [13]:
# The options for when we search for a new polytope given positivity certificates.
find_polytope_given_lagrangian_option = CspaceFreePolytope.FindPolytopeGivenLagrangianOptions()
find_polytope_given_lagrangian_option.solver_options = solver_options
find_polytope_given_lagrangian_option.ellipsoid_margin_cost = CspaceFreePolytope.EllipsoidMarginCost.kGeometricMean
find_polytope_given_lagrangian_option.search_s_bounds_lagrangians = True
find_polytope_given_lagrangian_option.ellipsoid_margin_epsilon = 1e-8
find_polytope_given_lagrangian_option.solver_id = solver_id

In [14]:
bilinear_alternation_options = CspaceFreePolytope.BilinearAlternationOptions()
bilinear_alternation_options.max_iter = 20 # Setting this to a high number will lead to more fill
bilinear_alternation_options.convergence_tol = 1e-3
bilinear_alternation_options.find_polytope_options = find_polytope_given_lagrangian_option

# The options for when we search for new planes and positivity certificates given the polytopes
# find_separation_certificate_given_polytope_options = CspaceFreePolytope.FindSeparationCertificateGivenPolytopeOptions()
# find_separation_certificate_given_polytope_options.num_threads = -1
# Parallelism 	parallelism {Parallelism::Max()}
bilinear_alternation_options.find_lagrangian_options.verbose = True
bilinear_alternation_options.find_lagrangian_options.solver_options = solver_options
bilinear_alternation_options.find_lagrangian_options.ignore_redundant_C = False
bilinear_alternation_options.find_lagrangian_options.solver_id = solver_id

In [15]:
binary_search_options = CspaceFreePolytope.BinarySearchOptions()
binary_search_options.scale_min = 1
binary_search_options.scale_max = 50
binary_search_options.max_iter = 100
binary_search_options.find_lagrangian_options.verbose = True
binary_search_options.find_lagrangian_options.solver_options = solver_options
binary_search_options.find_lagrangian_options.ignore_redundant_C = False
binary_search_options.find_lagrangian_options.solver_id = solver_id
# binary_search_options.find_lagrangian_options = find_separation_certificate_given_polytope_options

In [16]:
from time import perf_counter

In [17]:
bilinear_alternation_results_by_seed_point = dict.fromkeys([tuple(s) for s in seed_points])
times_per_region = []

In [ ]:
# We grow certified regions around each seedpoint using bilinear alternation.
for i, (s, (initial_region, color)) in enumerate(zip(seed_points, initial_regions)):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    bilinear_alternation_options.find_polytope_options.s_inner_pts = s
    t_start = time.perf_counter()
    result = cspace_free_polytope.SearchWithBilinearAlternation(set(), initial_region.A(),
                                                                initial_region.b(),
                                                                bilinear_alternation_options)
    t_end = time.perf_counter()
    times_per_region.append(t_end - t_start)
    bilinear_alternation_results_by_seed_point[tuple(s)] = [(cert.certified_polytope(),
                                                             cert, color) for cert in result]

In [ ]:
# visualize the regions and corresponding certificates
for i, result in enumerate(bilinear_alternation_results_by_seed_point.values()):
    group_name = f"/bil_alt/seed_point_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                               wireframe = False, opacity = 0.2)

In [ ]:
# We grow certified regions around each seedpoint using binary search.
binary_search_results_by_seed_point = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, (initial_region, color)) in enumerate(zip(seed_points, initial_regions)):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    time.sleep(0.2)
#     print(initial_region.A(), initial_region.b())
    cert = cspace_free_polytope.BinarySearch(set(),initial_region.A(),
                                               initial_region.b(),s,binary_search_options)
    print(cert)
    binary_search_results_by_seed_point[tuple(s)] = [(cert.certified_polytope(), cert, color)]                            

In [ ]:
# visualize the regions and corresponding certificates
for i, result in enumerate(binary_search_results_by_seed_point.values()):
    if result[0][0] is None:
        print(result)
    else:
        group_name = f"/bin_search/seed_point_{i}"
        visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                                   wireframe = False, opacity = 0.2)

In [ ]:
# some of my results were None because the seed points are already in collision

In [ ]:
# we use this to back off a bit from the binary search results for numerical reasons
def scale_polytope_about_point(hpoly, s, scale):
    if not hpoly.PointInSet(s):
        raise ValueError(f"The point s must be in the HPolyhedron")
    b_scaled = scale * hpoly.b() + (1 - scale) * hpoly.A() @ s
    return HPolyhedron(hpoly.A(), b_scaled)

In [ ]:
# Now we grow certified regions around each seedpoint using bilinear alternation 
# starting from the binary certified regions.

binary_and_bilinear_certified_regions = binary_search_results_by_seed_point.copy()
for i, (s, val) in enumerate(binary_and_bilinear_certified_regions.items()):
    (initial_region, cert, color) = val[0]
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    time.sleep(0.2)
    cur_s = np.array(s)
    bilinear_alternation_options.find_polytope_options.s_inner_pts = cur_s
    
    # We back off a little bit from the binary search solution so that we don't encounter numerical issues.
    initial_region = scale_polytope_about_point(initial_region, cur_s, 0.90)
    certificates = cspace_free_polytope.SearchWithBilinearAlternation(set(),
                                                                      initial_region.A(),
                                                                      initial_region.b(), 
                                                                      bilinear_alternation_options)
    binary_and_bilinear_certified_regions[s] += [(result.certified_polytope(), result, color) for result in certificates]

In [23]:
# visualize the regions and corresponding certificates
for i, result in enumerate(binary_and_bilinear_certified_regions.values()):
    group_name = f"/bin_then_bil/seed_point_{i}/"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                               wireframe = False, opacity = 0.2,
                                                               fill = False)
    visualizer.add_group_of_regions_and_certs_to_visualization([result[-1]], group_name+"/final" , 
                                                               wireframe = False, opacity = 0.2,
                                                               line_width = 1)

In [16]:
iris_regions = []
iris_ellipses = []

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.configuration_space_margin = 1e-3
iris_options.relative_termination_threshold = 0.001

context_for_iris = visualizer.task_space_diagram_context
for i, s in enumerate(seed_points):
    q = Ratfk.ComputeQValue(s, np.zeros((2,)))
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context_for_iris), q)
    r = IrisInRationalConfigurationSpace(plant, 
                                         plant.GetMyContextFromRoot(context_for_iris),
                                         q_star, iris_options)
    iris_regions.append((r, colors_dict[i]))
    iris_ellipses.append(r.MaximumVolumeInscribedEllipsoid())

INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 1
INFO:drake:IrisInConfigurationSpace: Terminating because the hyperellipsoid volume change 0.0012264641869944026 is below the threshold 0.02.
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 1
INFO:drake:IrisInConfigurationSpace iteration 2
INFO:drake:IrisInConfigurationSpace iteration 3
INFO:drake:IrisInConfigurationSpace: Terminating because the hyperellipsoid volume change 0.0014710271781924744 is below the threshold 0.02.
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 1
INFO:drake:IrisInConfigurationSpace: terminating iterations because the seed point is no longer in the region.
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 1
INFO:drake:IrisInConfigurationSpace iteration 2
INFO:drake:IrisInConfigurationSpace: Terminating because the hyperellipsoid vol

In [17]:
visualizer.add_group_of_regions_to_visualization(iris_regions, "/uncertified-iris", 
                                                            wireframe = False, opacity = 0.2)

In [18]:
binary_search_region_certificates_for_iris = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, (initial_region, color)) in enumerate(zip(seed_points, iris_regions)):
    print(f"starting seedpoint {i+1}/{len(iris_regions)}")
    time.sleep(0.2)    
    cert = cspace_free_polytope.BinarySearch(set(),
                                                    initial_region.A(),
                                                    initial_region.b(), 
                                                    initial_region.MaximumVolumeInscribedEllipsoid().center(), 
                                                    binary_search_options)
    binary_search_region_certificates_for_iris[tuple(s)] = [(cert.certified_polytope(), cert, color)]

starting seedpoint 1/5
starting seedpoint 2/5
starting seedpoint 3/5
starting seedpoint 4/5
starting seedpoint 5/5


In [19]:
for i, result in enumerate(binary_search_region_certificates_for_iris.values()):
    group_name = f"/certified-iris-bin_seed_point_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                            wireframe = False, opacity = 0.2)

In [20]:
# This plane certifies that the two tips of the flippers don't |intersect
visualizer.add_plane_indices_of_interest(118)

# re-display the sliders for convenience
for slider in sliders:
    display(slider)

{Variable('s[0]', Continuous): 0.5859170125984708, Variable('s[1]', Continuous): -0.24471670271446497}


RuntimeError: Monomial s[1] cannot be evaluated with the given environment which does not provide an entry for variable = s[1].

In [26]:
plane_indices = np.arange(0, len(cspace_free_polytope.separating_planes()))
last_plane_index = 0
cur_plane_index = 0
plane_indices_to_geometries = {v: k for k,v in cspace_free_polytope.map_geometries_to_separating_planes().items()}

visualizer.clear_plane_indices_of_interest()
while True:
    print(f"current plane index = {cur_plane_index}")
    
    last_plane_index = cur_plane_index
    valid_key_pressed = False
    while not valid_key_pressed:
        key_press = input("Enter d to increase plane_count, a to decrease, and e to exit...")
        if key_press in ["a", "d", "e"]:
            valid_key_pressed = True
    if key_press == "a":
        cur_plane_index = max(cur_plane_index - 1, min(plane_indices))
    elif key_press == "d":
        cur_plane_index = min(cur_plane_index + 1, max(plane_indices))
    elif key_press == "e":
        break
    visualizer.remove_plane_indices_of_interest(last_plane_index)
    visualizer.add_plane_indices_of_interest(cur_plane_index)
    cur_q = visualizer.plant.GetPositions(visualizer.plant_context)
    visualizer.show_res_q(cur_q)
    print()

current plane index = 0


KeyboardInterrupt: Interrupted by user

In [20]:
visualizer.region_certificate_groups

{'initial_regions': [(<pydrake.geometry.optimization.HPolyhedron at 0x7d1307a5ce70>,
   None,
   Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
  (<pydrake.geometry.optimization.HPolyhedron at 0x7d1307a2e270>,
   None,
   Rgba(r=0.118, g=0.533, b=0.898, a=0.2)),
  (<pydrake.geometry.optimization.HPolyhedron at 0x7d1307a5cf70>,
   None,
   Rgba(r=1.0, g=0.757, b=0.027, a=0.2)),
  (<pydrake.geometry.optimization.HPolyhedron at 0x7d1307a5d030>,
   None,
   Rgba(r=0.0, g=0.549, b=0.024, a=0.2)),
  (<pydrake.geometry.optimization.HPolyhedron at 0x7d1307a5d0b0>,
   None,
   Rgba(r=0.055, g=0.914, b=0.929, a=0.2))],
 '/uncertified-iris': [(<pydrake.geometry.optimization.HPolyhedron at 0x7d137d7c15f0>,
   None,
   Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
  (<pydrake.geometry.optimization.HPolyhedron at 0x7d137c1846b0>,
   None,
   Rgba(r=0.118, g=0.533, b=0.898, a=0.2)),
  (<pydrake.geometry.optimization.HPolyhedron at 0x7d1307aa7e30>,
   None,
   Rgba(r=1.0, g=0.757, b=0.027, a=0.2)),
  (<pydrake.g

In [23]:
[(group_name, region_and_cert_list) for group_name, region_and_cert_list in visualizer.region_certificate_groups.items()]

[('initial_regions',
  [(<pydrake.geometry.optimization.HPolyhedron at 0x7d1307a5ce70>,
    None,
    Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
   (<pydrake.geometry.optimization.HPolyhedron at 0x7d1307a2e270>,
    None,
    Rgba(r=0.118, g=0.533, b=0.898, a=0.2)),
   (<pydrake.geometry.optimization.HPolyhedron at 0x7d1307a5cf70>,
    None,
    Rgba(r=1.0, g=0.757, b=0.027, a=0.2)),
   (<pydrake.geometry.optimization.HPolyhedron at 0x7d1307a5d030>,
    None,
    Rgba(r=0.0, g=0.549, b=0.024, a=0.2)),
   (<pydrake.geometry.optimization.HPolyhedron at 0x7d1307a5d0b0>,
    None,
    Rgba(r=0.055, g=0.914, b=0.929, a=0.2))]),
 ('/uncertified-iris',
  [(<pydrake.geometry.optimization.HPolyhedron at 0x7d137d7c15f0>,
    None,
    Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
   (<pydrake.geometry.optimization.HPolyhedron at 0x7d137c1846b0>,
    None,
    Rgba(r=0.118, g=0.533, b=0.898, a=0.2)),
   (<pydrake.geometry.optimization.HPolyhedron at 0x7d1307aa7e30>,
    None,
    Rgba(r=1.0, g=0.757, b=

In [22]:
test = []
for group_name, region_and_cert_list in visualizer.region_certificate_groups.items():
    for i, (region, search_result, color) in enumerate(region_and_cert_list):
        test.append(search_result)

In [49]:
env = {var_s: val_s for var_s, val_s in zip(visualizer.rational_forward_kin.s(), s)}
print(env, test[-1].a()[118][0].indeterminates())
test[-1].a()[118][0].Evaluate(env)

{Variable('s[0]', Continuous): 0.5859170125984708, Variable('s[1]', Continuous): -0.24471670271446497} {s[0], s[1]}


RuntimeError: Monomial s[1] cannot be evaluated with the given environment which does not provide an entry for variable = s[1].

In [50]:
for i in env.keys():
    print(type(i))

<class 'pydrake.symbolic.Variable'>
<class 'pydrake.symbolic.Variable'>


In [51]:
binary_search_region_certificates_for_iris

{(0.0,
  0.0): [(<pydrake.geometry.optimization.HPolyhedron at 0x7ca74b156970>,
   Rgba(r=0.565, g=0.565, b=0.565, a=0.2))],
 (0.2027100355086725,
  0.2027100355086725): [(<pydrake.geometry.optimization.HPolyhedron at 0x7ca74b154cb0>,
   Rgba(r=0.118, g=0.533, b=0.898, a=0.2))],
 (-0.25534192122103627,
  0.25534192122103627): [(<pydrake.geometry.optimization.HPolyhedron at 0x7ca7b5b20570>,
   Rgba(r=1.0, g=0.757, b=0.027, a=0.2))],
 (0.25534192122103627,
  0.5994296231624898): [(<pydrake.geometry.optimization.HPolyhedron at 0x7ca74b14f530>,
   Rgba(r=0.0, g=0.549, b=0.024, a=0.2))],
 (0.5859170125984708,
  -0.24471670271446497): [(<pydrake.geometry.optimization.HPolyhedron at 0x7ca74b1484b0>,
   Rgba(r=0.055, g=0.914, b=0.929, a=0.2))]}

In [53]:
[tuple(s) for s in seed_points]

[(0.0, 0.0),
 (0.2027100355086725, 0.2027100355086725),
 (-0.25534192122103627, 0.25534192122103627),
 (0.25534192122103627, 0.5994296231624898),
 (0.5859170125984708, -0.24471670271446497)]

In [54]:
seed_points

array([[ 0.        ,  0.        ],
       [ 0.20271004,  0.20271004],
       [-0.25534192,  0.25534192],
       [ 0.25534192,  0.59942962],
       [ 0.58591701, -0.2447167 ]])

In [60]:
regions = []
for group_name, region_and_cert_list in visualizer.region_certificate_groups.items():
    regions.extend([l[0] for l in region_and_cert_list])

In [61]:
regions